In [44]:
import pandas as pd
import numpy as np
import seaborn as sns                       
import matplotlib.pyplot as plt           
%matplotlib inline     
sns.set(color_codes=True)

In [45]:
confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recoveries = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

In [46]:
recoveries

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/2/22,5/3/22,5/4/22,5/5/22,5/6/22,5/7/22,5/8/22,5/9/22,5/10/22,5/11/22
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
265,NaN,Winter Olympics 2022,39.904200,116.407400,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
266,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
267,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
confirmed = pd.melt(confirmed, id_vars=confirmed.columns[:4], 
                    value_vars = confirmed.columns[4:], 
                    var_name = 'date', value_name = 'confirmed')
deaths = pd.melt(deaths, id_vars=deaths.columns[:4], 
                 value_vars = deaths.columns[4:], 
                 var_name = 'date', value_name = 'deaths')
recoveries = pd.melt(recoveries, id_vars=recoveries.columns[:4],
                     value_vars = recoveries.columns[4:], 
                     var_name = 'date', value_name = 'recoveries')

In [48]:
deaths

,Province/State,Country/Region,Lat,Long,date,deaths
0,NaN,Afghanistan,33.939110,67.709953,1/22/20,0
1,NaN,Albania,41.153300,20.168300,1/22/20,0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0
...,...,...,...,...,...,...
238839,NaN,West Bank and Gaza,31.952200,35.233200,5/11/22,5657
238840,NaN,Winter Olympics 2022,39.904200,116.407400,5/11/22,0
238841,NaN,Yemen,15.552727,48.516388,5/11/22,2149
238842,NaN,Zambia,-13.133897,27.849332,5/11/22,3983


In [49]:
confirmed.columns = confirmed.columns.str.strip()

In [50]:
# Summarize confirmed and deaths data by date
confirmed_canada = confirmed[confirmed['Country/Region'] == 'Canada'].groupby('date').sum()[['confirmed']]
deaths_canada = deaths[deaths['Country/Region'] == 'Canada'].groupby('date').sum()[['deaths']]

# Extract columns from recoveries table
recoveries_canada = recoveries[recoveries['Country/Region'] == 'Canada']
canada_template = recoveries_canada[recoveries_canada.columns[:-1]].reset_index(drop=True)

# Join aggrregated confirmed and deaths data with extracted columns
confirmed_canada = canada_template.merge(confirmed_canada, how='inner', left_on='date', right_index=True)
deaths_canada = canada_template.merge(deaths_canada, how='inner', left_on='date', right_index=True)

# Add the agrregated data for Canada back to confirmed and deaths table
confirmed = confirmed[confirmed['Country/Region'] != 'Canada'].append(confirmed_canada)
deaths = deaths[deaths['Country/Region'] != 'Canada'].append(deaths_canada)



In [52]:
# Join confirmed, deaths and recoveries data together
data = confirmed.merge(deaths, how='inner',on=['Country/Region','Province/State','date']).merge(recoveries, how='inner',on=['Country/Region','Province/State','date'])
data['recoveries'] = data['recoveries'].astype("int")
data = data[['Province/State','Country/Region','date','Lat','Long','confirmed','deaths','recoveries']]

In [54]:
data.sample(10)

,Province/State,Country/Region,date,Lat,Long,confirmed,deaths,recoveries
70474,NaN,Uruguay,10/10/20,-32.522800,-55.765800,2268,50,1930
110094,NaN,Sierra Leone,3/7/21,8.460555,-11.779889,3920,79,2700
174915,"Bonaire, Sint Eustatius and Saba",Netherlands,11/4/21,12.178400,-68.238500,2468,19,0
103313,NaN,Indonesia,2/10/21,-0.789300,113.921300,1183555,32167,982972
104534,Tasmania,Australia,2/15/21,-42.882100,147.327200,234,13,221
65054,NaN,Poland,9/20/20,51.919400,19.145100,79240,2293,64302
34753,Sint Maarten,Netherlands,5/30/20,18.042500,-63.054800,77,15,60
115190,NaN,Solomon Islands,3/26/21,-9.645700,160.156200,18,0,16
1850,NaN,Ukraine,1/28/20,48.379400,31.165600,0,0,0
100533,NaN,Bulgaria,1/31/21,42.733900,25.485800,218748,9045,187052


In [56]:
population = pd.read_csv('population_by_country_2020.csv')
population

,Country (or dependency),Population (2020),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share
0,China,1440297825,0.39 %,5540090,153,9388211,-348399.0,1.7,38,61 %,18.47 %
1,India,1382345085,0.99 %,13586631,464,2973190,-532687.0,2.2,28,35 %,17.70 %
2,United States,331341050,0.59 %,1937734,36,9147420,954806.0,1.8,38,83 %,4.25 %
3,Indonesia,274021604,1.07 %,2898047,151,1811570,-98955.0,2.3,30,56 %,3.51 %
4,Pakistan,221612785,2.00 %,4327022,287,770880,-233379.0,3.6,23,35 %,2.83 %
...,...,...,...,...,...,...,...,...,...,...,...
230,Montserrat,4993,0.06 %,3,50,100,NaN,N.A.,N.A.,10 %,0.00 %
231,Falkland Islands,3497,3.05 %,103,0,12170,NaN,N.A.,N.A.,66 %,0.00 %
232,Niue,1628,0.68 %,11,6,260,NaN,N.A.,N.A.,46 %,0.00 %
233,Tokelau,1360,1.27 %,17,136,10,NaN,N.A.,N.A.,0 %,0.00 %


In [57]:
for c in data['Country/Region'].unique():
  if c not in population['Country (or dependency)'].unique():
    print(c)

Antarctica
Burma
Congo (Brazzaville)
Congo (Kinshasa)
Cote d'Ivoire
Czechia
Diamond Princess
Korea, South
Kosovo
MS Zaandam
Saint Kitts and Nevis
Saint Vincent and the Grenadines
Sao Tome and Principe
Summer Olympics 2020
Taiwan*
US
West Bank and Gaza
Winter Olympics 2022


In [58]:
country_mapper = {
    'Congo (Brazzaville)': 'Congo',
    'Congo (Kinshasa)': 'Congo',
    "Cote d'Ivoire": "Côte d'Ivoire",
    'Czechia': 'Czech Republic (Czechia)',
    'Korea, South': 'South Korea',
    'Saint Vincent and the Grenadines': 'St. Vincent & Grenadines',
    'Taiwan*': 'Taiwan',
    'US': 'United States',
    'West Bank and Gaza': 'Israel',
    'Saint Kitts and Nevis': 'Saint Kitts & Nevis',
    'Burma': 'Myanmar',
    'Sao Tome and Principe': 'Sao Tome & Principe'
}

data['Country/Region'] = data['Country/Region'].replace(country_mapper)
data.index = data['Country/Region']

In [60]:
data.to_csv('covid19.csv')